In [17]:
!export CUDA_VISIBLE_DEVICES=1
from vae_dist.scripts.train_escnn import R3Net
from escnn import gspaces, nn 


In [20]:
group = gspaces.flipRot3dOnR3(maximum_frequency=10) 
input_out_reps = 3*[group.trivial_repr]
kernel_size = 1
feat_type_in  = nn.FieldType(group,  input_out_reps) 
feat_type_out = nn.FieldType(group,  input_out_reps)   
model = R3Net.load_from_checkpoint(
        "./lightning_logs/version_103/checkpoints/epoch-99-step=2000.ckpt",
        group = group,
        feat_type_in = feat_type_in, 
        feat_type_out = feat_type_out, 
        )



FileNotFoundError: [Errno 2] No such file or directory: '/home/santiagovargas/dev/vae_dist/vae_dist/scripts/./lightning_logs/version_103/checkpoints/epoch-99-step=2000.ckpt'